# Generate synthetic data
Generating synthetic data using public libraries for the 'adult' dataset.

## Load libraries and define settings

In [1]:
# general dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
from pathlib import Path
import warnings

In [2]:
# package
from crnsynth.generators.marginal_generator import MarginalGenerator
from crnsynth.generators.privbayes import PrivBayes

from crnsynth.processing import preprocessing, postprocessing
from crnsynth.synthesization import synthesization, pipeline
from crnsynth.processing.generalization import NumericGeneralizationMech

In [4]:
from examples.synthesization import adult_synthesis

In [5]:
# ignore deprecation warning
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# autoreload changes from local files
%load_ext autoreload
%autoreload 2

# pandas show full output
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)

In [6]:
# privacy budget for differentially private algorithms
EPSILON = 1

# other constants
RANDOM_STATE = 42
VERBOSE = 1

## Load and process dataset

In [7]:
df_adult = pd.read_csv(adult_synthesis.PATH_ADULT)
df_adult.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Generate
Here we show how step by step how to generate synthetic data.

In [8]:
# define your generator - in this case the simple marginal generator
generator = MarginalGenerator(epsilon=EPSILON)

# train the generator on the input data
generator.fit(df_adult)

# generate synthetic data with a desired number of rows
df_synth_marginal = generator.generate(n_records=1000)
df_synth_marginal.head()

Marginal fitted: age
Marginal fitted: workclass
Marginal fitted: fnlwgt
Marginal fitted: education
Marginal fitted: education-num
Marginal fitted: marital-status
Marginal fitted: occupation
Marginal fitted: relationship
Marginal fitted: race
Marginal fitted: sex
Marginal fitted: capital-gain
Marginal fitted: capital-loss
Marginal fitted: hours-per-week
Marginal fitted: native-country
Marginal fitted: income
Column sampled: age
Column sampled: workclass
Column sampled: fnlwgt
Column sampled: education
Column sampled: education-num
Column sampled: marital-status
Column sampled: occupation
Column sampled: relationship
Column sampled: race
Column sampled: sex
Column sampled: capital-gain
Column sampled: capital-loss
Column sampled: hours-per-week
Column sampled: native-country
Column sampled: income


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,21,Private,199725,Prof-school,9,Never-married,Adm-clerical,Unmarried,White,Male,0,0,80,United-States,<=50K
1,22,Private,252565,Masters,5,Never-married,Craft-repair,Husband,White,Male,0,0,50,United-States,<=50K
2,35,Private,162840,HS-grad,10,Never-married,Handlers-cleaners,Husband,White,Female,0,0,40,Japan,<=50K
3,66,Private,150080,Assoc-acdm,14,Married-civ-spouse,Tech-support,Husband,White,Female,0,0,60,United-States,>50K
4,19,Private,258498,Assoc-voc,10,Never-married,Other-service,Husband,White,Male,0,0,28,United-States,<=50K


## Improving the quality of the synthetic data
Additional processing steps can be added to the generation process to improve the quality of the synthetic data.

In [9]:
def preprocess_real_data(data_real: pd.DataFrame) -> pd.DataFrame:
    """Process data before fitting the generator."""
    # reduce columns
    columns_subset = [
        "age",
        "workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "hours-per-week",
        "native-country",
        "income",
    ]
    data_real = data_real[columns_subset]
    return data_real

# create copy to retain original data format
df = df_adult.copy()

# split in training and holdout set - train is used for fitting the generator, holdout for evaluation
df_train, df_holdout = preprocessing.split_train_holdout(df, target_column='income', holdout_size=0.2, random_state=RANDOM_STATE)

# preprocess the data
df_train = preprocess_real_data(df_train)
df_train.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
15738,32,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K
27985,43,Private,Masters,Never-married,Exec-managerial,Not-in-family,White,Female,45,United-States,<=50K
30673,20,?,HS-grad,Never-married,?,Not-in-family,White,Female,28,United-States,<=50K
9505,40,Local-gov,Some-college,Divorced,Transport-moving,Unmarried,White,Male,40,United-States,<=50K
26417,24,Private,Bachelors,Never-married,Prof-specialty,Own-child,White,Male,40,United-States,<=50K


Reducing the dimensionality of the data can help the generator to learn the underlying distribution better. We can generalize the data by binning the numerical columns.

In [10]:
# specify generalization mechanisms
generalizers = [
    NumericGeneralizationMech(column="age", epsilon=0.05, bins=5, bounds=adult_synthesis.AGE_BOUNDS),
    NumericGeneralizationMech(column="hours-per-week", epsilon=0.05, bins=5, bounds=adult_synthesis.HOURS_PER_WEEK_BOUNDS),
]

# apply generalization mechanisms
df_train_input, generalizers = preprocessing.generalize_data(df_train, generalizers=generalizers)
df_train_input.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
15738,1,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,2,United-States,>50K
27985,1,Private,Masters,Never-married,Exec-managerial,Not-in-family,White,Female,2,United-States,<=50K
30673,0,?,HS-grad,Never-married,?,Not-in-family,White,Female,1,United-States,<=50K
9505,1,Local-gov,Some-college,Divorced,Transport-moving,Unmarried,White,Male,1,United-States,<=50K
26417,0,Private,Bachelors,Never-married,Prof-specialty,Own-child,White,Male,1,United-States,<=50K


Fit the generator on the preprocessed data and generate synthetic data.

In [11]:
generator=PrivBayes(epsilon=EPSILON, verbose=VERBOSE)
generator.fit(df_train_input)

1/11 - Root of network: hours-per-week

2/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 10
Selected attribute: 'occupation' - with parents: ('hours-per-week',)

3/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 14
Selected attribute: 'workclass' - with parents: ('occupation',)

4/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 19
Selected attribute: 'sex' - with parents: ('hours-per-week', 'occupation')

5/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 18
Selected attribute: 'relationship' - with parents: ('hours-per-week', 'sex')

6/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 24
Selected attribute: 'marital-status' - with parents: ('relationship', 'sex')

7/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 28
Selected attribut

/Users/dknoors/miniconda3/envs/crnsynth/lib/python3.9/site-packages/thomas/core/factors/cpt.py:65: RuntimeWarning: invalid value encountered in divide
  self.values = self.values / row_sum
/Users/dknoors/miniconda3/envs/crnsynth/lib/python3.9/site-packages/thomas/core/factors/cpt.py:65: RuntimeWarning: invalid value encountered in divide
  self.values = self.values / row_sum


In [12]:
df_synth = generator.generate(n_records=df.shape[0])
df_synth.head()

Number of records generated: 2986 / 32561

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Number of records generated: 11883 / 32561

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Number of records generated: 20965 / 32561

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Number of records generated: 30306 / 32561

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



The synthetic data still has generalized values. We can reverse the generalization by sampling within the bounds of the generalized value to retrieve the original data structure.

In [13]:
df_synth_inv, generalizers = postprocessing.reverse_generalization(df_synth, generalizers)
df_synth_inv.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,66,Private,Bachelors,Never-married,Other-service,Own-child,White,Female,21,United-States,<=50K
1,70,Local-gov,HS-grad,Divorced,Transport-moving,Own-child,White,Female,34,United-States,<=50K
2,42,Private,Some-college,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,Male,66,United-States,>50K
3,31,Private,HS-grad,Never-married,Transport-moving,Not-in-family,White,Female,23,United-States,<=50K
4,22,Self-emp-inc,Some-college,Married-civ-spouse,Sales,Husband,White,Male,15,United-States,>50K


Addtional postprocessing steps can be added to the pipeline to improve the quality of the synthetic data.

In [14]:
def postprocess_synth_data(data_synth: pd.DataFrame) -> pd.DataFrame:
    """Post-process the synthetic data."""
    # add identifier to notify user that this is synthetic data (fake)
    id_string = 'FAKE_PERSON_'
    data_synth['id'] = [id_string + str(i) for i in np.arange(1, data_synth.shape[0]+1)]
    
    # make id column first
    cols = data_synth.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    data_synth = data_synth[cols]
    return data_synth

df_synth_final = postprocess_synth_data(df_synth_inv)
df_synth_final.head()

,id,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,FAKE_PERSON_1,66,Private,Bachelors,Never-married,Other-service,Own-child,White,Female,21,United-States,<=50K
1,FAKE_PERSON_2,70,Local-gov,HS-grad,Divorced,Transport-moving,Own-child,White,Female,34,United-States,<=50K
2,FAKE_PERSON_3,42,Private,Some-college,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,Male,66,United-States,>50K
3,FAKE_PERSON_4,31,Private,HS-grad,Never-married,Transport-moving,Not-in-family,White,Female,23,United-States,<=50K
4,FAKE_PERSON_5,22,Self-emp-inc,Some-college,Married-civ-spouse,Sales,Husband,White,Male,15,United-States,>50K


## Synth Pipeline
All these steps can be combined in a single function to create a pipeline for generating synthetic data. Note this function provides a consistent structure for creating synthetic data, but can also serve as an example for creating your own pipeline in case you need a bit more flexibility.

In [15]:
# create a synthetic pipeline
df = df_adult.copy()
generator=PrivBayes(epsilon=EPSILON, verbose=VERBOSE)
generalizers = [
    NumericGeneralizationMech(column="age", epsilon=0.05, bins=5, bounds=adult_synthesis.AGE_BOUNDS, inverse='truncated_normal', random_state=RANDOM_STATE),
    NumericGeneralizationMech(column="hours-per-week", epsilon=0.05, bins=5, bounds=adult_synthesis.HOURS_PER_WEEK_BOUNDS, inverse='truncated_normal', random_state=RANDOM_STATE)
]
output = pipeline.run_synth_pipeline(
    data_real=df,
    generator=generator,
    preprocess_func=adult_synthesis.preprocess_real_data,
    generalizers=generalizers,
    postprocess_func=adult_synthesis.postprocess_synthetic_data,
    holdout_size=0.2,
    target_column='income',
    random_state=RANDOM_STATE,
    n_records=1000,
    output_keys= ["train", "holdout", "synth", "generators"],
    verbose=VERBOSE
)
output['synth'].head()

Fitting generator PrivBayes({'epsilon': 1, 'verbose': 1, 'theta_usefulness': 4, 'epsilon_split': 0.3, 'score_function': 'R', 'network_init': None, 'n_cpus': None}) on input data
1/11 - Root of network: native-country

2/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 10
Selected attribute: 'workclass' - with parents: None

3/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 11
Selected attribute: 'occupation' - with parents: ('workclass',)

4/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 17
Selected attribute: 'relationship' - with parents: ('occupation',)

5/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 20
Selected attribute: 'marital-status' - with parents: ('relationship',)

6/11 - Evaluating next attribute to add to network
Number of AttributeParentPair candidates: 25
Selected attribute: 'sex' - with parents: (

/Users/dknoors/miniconda3/envs/crnsynth/lib/python3.9/site-packages/thomas/core/factors/cpt.py:65: RuntimeWarning: invalid value encountered in divide
  self.values = self.values / row_sum
/Users/dknoors/miniconda3/envs/crnsynth/lib/python3.9/site-packages/thomas/core/factors/cpt.py:65: RuntimeWarning: invalid value encountered in divide
  self.values = self.values / row_sum


Number of records generated: 1000 / 1000
Synthetic Data Generated



,age,hours-per-week,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
0,24,30,Private,HS-grad,Never-married,Adm-clerical,Unmarried,Black,Female,United-States,<=50K
1,45,39,Private,Prof-school,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,>50K
2,55,52,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,>50K
3,53,34,Private,Prof-school,Married-civ-spouse,Craft-repair,Husband,White,Male,United-States,>50K
4,48,42,Private,9th,Divorced,Exec-managerial,Unmarried,White,Female,United-States,<=50K
